In [1]:
!pip install protlearn
!pip install rdkit
!pip install catboost
!pip install mordred
!pip install minepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 46.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 7.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.5 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 s

In [2]:
import numpy as np
import urllib.request
import pandas as pd
from rdkit import rdBase, Chem, DataStructs 
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem import Draw
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from protlearn.preprocessing import remove_unnatural
from mordred import Calculator, descriptors
from sklearn.decomposition import PCA

In [3]:
from protlearn.features import *
import numpy as np
datalist=[]
datalisttest=[]
f1=open("/content/drive/MyDrive/traindata.csv","r")
f2=open("/content/drive/MyDrive/testdata.csv","r")

for line in f1:
  line=line.replace("\n","")
  line=line.split(",")
  datalist.append(line)
f1.close()
for line in f2:
  line=line.replace("\n","")
  line=line.split(",")
  datalisttest.append(line)
f2.close()
print(len(datalist))
print(datalisttest)

23818
[['CC1=C(/C=C/C(C)=C/C=C/C(C)=C/C(=O)O)C(C)(C)CCC1', 'MLFESFDLVSALATLAACLVSMALLLAVPQQLWQLRWTATRDKSCKLPMPKGSMGFPIIGETCHWFFQGAGFHASRRQKYGNVFKTHLLGRPLIRVTGAENVRKVLMGEHSLVTVDWPQSTSTLLGPNSLANSIGDIHRKRRKIFAKVFSHEALESYLPKIQQVIQETLRVWSSNPDPINVYRESQRLSFNMAVRVLLGFRIPEEEMHCLFSTFQEFVENVFSLPIDLPFSGYRKGIRARDSLQKSIEKAIREKPLHTQGKDYTDALDVLLESAKENNTELTMQELKESTIELIFAAFATTASASTSLVMQLLRHPAVLEKLREELRSCGLLHDGCLCQGELRLDSIISLKYLDCVIKEVLRLFAPVSGGYRIATQTFELDGVQVPKGWSVMYSIRDTHDTSAVFKDVEAFDPDRFSPERSEDREGRFHYLPFGGGVRSCLGKQLATLFLKLLAVELAGGSRFELSTRTFPRMISVPVVHPTDGLRVKFFGLDSNQNQIMAKSDEMLDATV', '1'], ['c1ccc2c(c1)ccc1nc3ccc4ccccc4c3cc12', 'MDPFVVLVLCLSCLLLLSIWRQSSGRGKLPPGPTPLPVIGNILQIDIKDVSKSLTNLSKIYGPVFTLYFGLERMVVLHGYEVVKEALIDLGEEFSGRGHFPLAERANRGFGIVFSNGKRWKEIRRFSLMTLRNFGMGKRSIEDRVQEEARCLVEELRKTKASPCDPTFILGCAPCNVICSIIFQKRFDYKDQQFLNLMEKLNENIRIVSTPWIQICNNFPTIIDYFPGTHNKLLKNLAFMESDILEKVKEHQESMDINNPRDFIDCFLIKMEKEKQNQQSEFTIENLVITAADLLGAGTETTSTTLRYALLLLLKHPEVTAKVQEEIERVIGRNRSPCMQDRGHMPYTDAVVHEVQRYIDLIPTSLPHAVTCDVKFRNYL

In [4]:
ori_protein_seq=[data[1] for data in datalist]
ori_smiles_list=[data[0] for data in datalist]
ori_labels=[eval(data[2]) for data in datalist]

ori_protein_seq_test=[data[1] for data in datalisttest]
ori_smiles_list_test=[data[0] for data in datalisttest]
ori_labels_test=[eval(data[2]) for data in datalisttest]

dataunnatural=[]
error=[]
dataunnatural_test=[]
error_test=[]
for i in range(len(ori_protein_seq)):
  try:
    seq= remove_unnatural([ori_protein_seq[i]])
    if seq:
      continue
    else:
      dataunnatural.append(i)
  except:
      error.append(i)
      continue
print(len(dataunnatural))
print(error)

for i in range(len(ori_protein_seq_test)):
  try:
    seq= remove_unnatural([ori_protein_seq_test[i]])
    if seq:
      continue
    else:
      dataunnatural_test.append(i)
  except:
      error_test.append(i)
      continue

print(len(dataunnatural_test))
print(error_test)

protein_seq_train=[]
smiles_list_train=[]
labels_train=[]
for i in range(len(ori_protein_seq)):
  if len(ori_protein_seq[i])>=50:
    mol=Chem.MolFromSmiles(ori_smiles_list[i])
    if mol:
      if (i in dataunnatural) or (i in error):
        continue
      else:
        protein_seq_train.append(ori_protein_seq[i])
        smiles_list_train.append(ori_smiles_list[i])
        labels_train.append(ori_labels[i])
    else:
      print("error smiles id",i)
      print("error smiles",ori_smiles_list[i])
print(len(protein_seq_train))
print(len(smiles_list_train))
print(len(labels_train))

protein_seq_test=[]
smiles_list_test=[]
labels_test=[]
for i in range(len(ori_protein_seq_test)):
  if len(ori_protein_seq_test[i])>=50:
    mol=Chem.MolFromSmiles(ori_smiles_list_test[i])
    if mol:
      if (i in dataunnatural_test) or (i in error_test):
        continue
      else:
        protein_seq_test.append(ori_protein_seq_test[i])
        smiles_list_test.append(ori_smiles_list_test[i])
        labels_test.append(ori_labels_test[i])
    else:
      print("error test smiles id",i)
      print("error test smiles",ori_smiles_list_test[i])
print(len(protein_seq_test))
print(len(smiles_list_test))
print(len(labels_test))
protein_seq=protein_seq_train+protein_seq_test
smiles_list=smiles_list_train+smiles_list_test
labels=labels_train+labels_test

65
[11704, 18365]
0
[]


[12:36:39] SMILES Parse Error: unclosed ring for input: 'CC(C)=CCC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/Cc1cc(C(=O)O)ccc5O'


error smiles id 2522
error smiles CC(C)=CCC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/Cc1cc(C(=O)O)ccc5O


[12:36:40] SMILES Parse Error: unclosed ring for input: 'CC(C)=CCC/C(C)=C/CC/C(C)=C/Cc1c(O)cc(C)c(C(=O)O)c2O'
[12:36:40] non-ring atom 1 marked aromatic


error smiles id 5539
error smiles CC(C)=CCC/C(C)=C/CC/C(C)=C/Cc1c(O)cc(C)c(C(=O)O)c2O
error smiles id 5779
error smiles Cc(=O)O[C@H]1C[C@@H](C)[C@@H]2Cc[C@@]3(C)Cc(C)(C)[C@H]1[C@]23O


...

error smiles id 12423
error smiles CC(C)=CCC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/Cc1cc(C(=O)O)ccc3O


[12:36:42] Can't kekulize mol.  Unkekulized atoms: 15 16 18 19 21


error smiles id 13650
error smiles CC(C)=CCC/C(C)=C/CC/C(C)=C/Cc1c(O)cc(C)c(C=O)C1o


[12:36:42] SMILES Parse Error: unclosed ring for input: 'CC(C)=CCC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/Cc1cc(C(=O)O)ccc4O'
[12:36:43] Can't kekulize mol.  Unkekulized atoms: 15
[12:36:43] SMILES Parse Error: unclosed ring for input: 'CC(C)=CCC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/Cc1cc(C(=O)O)ccc6O'


error smiles id 14698
error smiles CC(C)=CCC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/Cc1cc(C(=O)O)ccc4O
error smiles id 16069
error smiles C[C@H](C=O)Ccc[C@@H](C)[C@H]1Cc[C@H]2[C@@H]3Ccc4=Cc(=O)Cc[C@]4(C)[C@H]3Cc[C@@]21C
error smiles id 16767
error smiles CC(C)=CCC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/CC/C(C)=C/Cc1cc(C(=O)O)ccc6O
23736
23736
23736
200
200
200


In [5]:
prolength=length(protein_seq)
proacc,aa = aac(protein_seq)
aaind, inds = aaindex1(protein_seq, standardize='zscore')
tri, ngrams = ngram(protein_seq, n=2)
ent = entropy(protein_seq)
atoms, bonds = atc(protein_seq)
ck, pairs = cksaap(protein_seq)
ctd_arr, ctd_desc = ctd(protein_seq)
c, desc = ctdc(protein_seq)
t, desct = ctdt(protein_seq)
d, descd = ctdd(protein_seq)
mb = moreau_broto(protein_seq)
moranI = moran(protein_seq)
gearyC = geary(protein_seq)
paac_comp, descpaac = paac(protein_seq)
apaac_comp, descapaac = apaac(protein_seq)
sw, g = socn(protein_seq)
swqso, gqso, descqso = qso(protein_seq, d=3, remove_zero_cols=True)
proteinfeatures = np.concatenate([prolength, proacc, aaind, tri, ent, atoms, bonds, ck, ctd_arr, c, t, d, mb, moranI, gearyC, paac_comp, apaac_comp, sw , g, swqso ,gqso], axis=1)
print(proteinfeatures.shape)
np.savetxt('/content/drive/MyDrive/proteinfeatures.csv',proteinfeatures, delimiter=",")

KeyboardInterrupt: ignored

In [ ]:
calc = Calculator(descriptors)
mol_descriptors=[]
desciptors_safe=[]
for i in range(len(smiles_list)):
  try:
    mol=Chem.MolFromSmiles(smiles_list[i])
    fingerprint=calc(mol)[:]
    mol_descriptors.append(fingerprint)
    desciptors_safe.append(i)
  except:
        print("Error",i)
        continue
mol_descriptors=np.array(mol_descriptors)

mol_MACCSfingerprints_safe = []
mol_MACCSfingerprints=[]
for i in range(len(smiles_list)):
    try:
        mol=Chem.MolFromSmiles(smiles_list[i])
        fingerprint = [x for x in Chem.MACCSkeys.GenMACCSKeys(mol)]
        mol_MACCSfingerprints.append(fingerprint)
        mol_MACCSfingerprints_safe.append(i)
    except:
        print("Error", i)
        continue
mol_MACCSfingerprints=np.array(mol_MACCSfingerprints)

mol_Topolfingerprints_safe = []
mol_Topolfingerprints=[]
for i in range(len(smiles_list)):

        mol=Chem.MolFromSmiles(smiles_list[i])
        fingerprint = [int(x) for x in Chem.RDKFingerprint(mol)]
        mol_Topolfingerprints.append(fingerprint)
        mol_Topolfingerprints_safe.append(i)

mol_Topolfingerprints=np.array(mol_Topolfingerprints)

mol_morganfingerprint=[]
safe = []
for i in range(len(smiles_list)):
    try:
        mol=Chem.MolFromSmiles(smiles_list[i])
        fingerprint = [x for x in AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048)]
        mol_morganfingerprint.append(fingerprint)
        safe.append(i)
    except:
        print("Error",i)
        continue
mol_morganfingerprint=np.array(mol_morganfingerprint)
    
mol_Avalonfingerprint=[]
Avalonsafe=[]
for i in range(len(smiles_list)):
    try:
        mol=Chem.MolFromSmiles(smiles_list[i])
        fingerprint = [x for x in pyAvalonTools.GetAvalonFP(mol)]
        mol_Avalonfingerprint.append(fingerprint)
        Avalonsafe.append(i)
    except:
        print("Error",i)
        continue
mol_Avalonfingerprint=np.array(mol_Avalonfingerprint)

chemfeatures=np.concatenate([mol_descriptors,mol_MACCSfingerprints,mol_Topolfingerprints,mol_morganfingerprint,mol_Avalonfingerprint],axis=1)

print(mol_MACCSfingerprints.shape)
print(mol_Topolfingerprints.shape)
print(mol_morganfingerprint.shape)
print(mol_Avalonfingerprint.shape)
print(mol_descriptors.shape)
print(chemfeatures.shape)
np.savetxt('/content/drive/MyDrive/chemfeatures.csv', chemfeatures, delimiter=",")

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


(23936, 167)
(23936, 2048)
(23936, 2048)
(23936, 512)
(23936, 1826)
(23936, 6601)


In [6]:
chemdata=[]

f1=open("/content/drive/MyDrive/chemfeatures.csv","r")
for i in f1:
  i=i.replace("\n","")
  i=i.split(",")
  chemdata.append(i[:167]+i[4263:])
for i in range(len(chemdata)):
  for j in range(len(chemdata[i])):
    if chemdata[i][j]=="nan":
      chemdata[i][j]=0
    else:
      chemdata[i][j]=float(chemdata[i][j])
chemfeatures=np.array(chemdata)
chemdata=[]
print("chem",chemfeatures.shape)
proteinfeatures=np.loadtxt("/content/drive/MyDrive/proteinfeatures.csv",delimiter=",")
labels=np.array(labels)
print("labels",labels.shape)

chem (23936, 2505)
labels (23936,)


In [8]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, roc_auc_score

data_x=np.concatenate([proteinfeatures,chemfeatures],axis=1)
data_y=labels 
print(data_x.shape)
print(data_y.shape)
proteinfeatures=[]
chemfatures=[]

(23936, 4764)
(23936,)


In [9]:
#PCA
pca=PCA(n_components=4,svd_solver="full")
pca_model=pca.fit(data_x)
data_x_pca=pca_model.fit_transform(data_x)
print(pca_model.explained_variance_ratio_)
print(data_x_pca.shape)
X_train, X_test, y_train, y_test = train_test_split(data_x_pca,data_y,test_size=0.2)

[9.99988857e-01 1.07939203e-05 2.18710490e-07 2.88340208e-08]
(23936, 4)


In [ ]:
#tree_important
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
rf=RandomForestClassifier(max_samples=5000)
rf=rf.fit(data_x,data_y)
print(rf.feature_importances_)
model = SelectFromModel(rf, prefit=True)
data_x_tree_importance=model.transform(data_x)
print(data_x_tree_importance)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(data_x, data_y)
model = SelectFromModel(lsvc, prefit=True)
data_x_l1 = model.transform(data_x)
data_x_l1.shape

In [ ]:
#mic
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from minepy import MINE
from sklearn.feature_selection import SelectKBest
#由于MINE的设计不是函数式的，定义mic方法将其为函数式的，返回一个二元组，二元组的第2项设置成固定的P值0.5
def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return (m.mic(), 0.5)
# 选择K个最好的特征，返回特征选择后的数据


data_x_mic=SelectKBest(lambda x, Y: np.array(map(lambda x:mic(x, Y), X.T)).T, k=2000).fit_transform(data_x,data_y)

In [13]:
#xgboost
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',   
    'gamma': 0.1,
    'max_depth': 7,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'eta': 0.001,
    'seed': 1000,
    'nthread': 4,
    "learning_rate": 0.05
}
t0 = time.time()
# plst = params.items()
num_rounds = 500
dtrain = xgb.DMatrix(X_train, y_train)
model_xgb = xgb.train(params, dtrain, num_rounds)
print('training spend {} seconds'.format(time.time()-t0))
# 对测试集进行预测
t1 = time.time()
dtest = xgb.DMatrix(X_test)
y_pred = model_xgb.predict(dtest)
print('testing spend {} seconds'.format(time.time()-t1))
y_pred_train = model_xgb.predict(dtrain)
print(roc_auc_score(y_train, y_pred_train))
print(roc_auc_score(y_test, y_pred))

training spend 7.743022441864014 seconds
testing spend 0.13429546356201172 seconds
0.9829492700559732
0.9549793763059322


In [14]:
from sklearn.model_selection import cross_val_score 
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',   
    'gamma': 0.1,
    'max_depth': 7,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'eta': 0.001,
    'seed': 1000,
    'nthread': 4,
    "learning_rate": 0.05
}
model = xgb.XGBClassifier(booster="gbtree",objective='binary:logistic',gamma=0.1,max_depth=7,subsample=0.7,colsample_bytree=0.7,min_child_weight=3,eta=0.001,learning_rate=0.05,nthread=4)

accuracy_scores = cross_val_score(model, data_x_pca, data_y, cv=10, scoring='accuracy')
f1_scores=cross_val_score(model, data_x_pca, data_y, cv=10, scoring='f1')
roc_auc_scores=cross_val_score(model, data_x_pca, data_y, cv=10, scoring='roc_auc')
print(accuracy_scores)
print("accuracy_scores_mean",accuracy_scores.mean())
print(f1_scores)
print("f1_scores_mean",f1_scores.mean())
print(roc_auc_scores)
print("roc_auc_scores_mean",roc_auc_scores.mean())

[0.90768588 0.90685046 0.91353383 0.91436926 0.91186299 0.91645781
 0.90722942 0.90221479 0.91475136 0.89928959]
accuracy_scores_mean 0.9094245398982903
[0.91732136 0.91638545 0.92244286 0.92301915 0.92058713 0.92509363
 0.91647856 0.91129644 0.92382375 0.90860827]
f1_scores_mean 0.918505659674748
[0.95717559 0.95257363 0.96032748 0.95802505 0.95604355 0.96128998
 0.95632962 0.95884763 0.96002369 0.95612542]
roc_auc_scores_mean 0.9576761633623656


In [ ]:
d_train = lgb.Dataset(X_train, label=y_train)
params = {"max_depth": 5, "learning_rate" : 0.05, "num_leaves": 500,  "n_estimators": 300}

t0 = time.time()
model_lgb = lgb.train(params, d_train)
print('training spend {} seconds'.format(time.time()-t0))
t1 = time.time()
y_pred = model_lgb.predict(X_test)
print('testing spend {} seconds'.format(time.time()-t1))
y_pred_train = model_lgb.predict(X_train)
print(roc_auc_score(y_train, y_pred_train))
print(roc_auc_score(y_test, y_pred))

In [ ]:
cat_features_index = [0,1,2,3,4,5,6]
t0 = time.time()
model_cb = cb.CatBoostClassifier(eval_metric="AUC", one_hot_max_size=50, 
                            depth=6, iterations=300, l2_leaf_reg=1, learning_rate=0.1)
model_cb.fit(X_train,y_train, cat_features= cat_features_index)
print('training spend {} seconds'.format(time.time()-t0))
t1 = time.time()
y_pred = model_cb.predict(X_test)
print('testing spend {} seconds'.format(time.time()-t1))
y_pred_train = model_cb.predict(X_train)
print(roc_auc_score(y_train, y_pred_train))
print(roc_auc_score(y_test, y_pred))

In [ ]:
### RandomSearch
from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform
model = xgb.XGBClassifier()
param_lst = {'max_depth': [3,5,7], 
                 'min_child_weight': [1,3,6], 
                 'n_estimators': [100,200,300],
                 'learning_rate': [0.01, 0.05, 0.1]
                }
t0 = time.time()
random_search = RandomizedSearchCV(model, param_lst, random_state=0)
random_search.fit(X_train, y_train)
print(random_search.best_params_)

print('randomsearch for xgb spend', time.time()-t0, 'seconds.')

In [ ]:
### GridSearch
from sklearn.model_selection import GridSearchCV
model = xgb.XGBClassifier()
param_lst = {"max_depth": [3,5,7],
              "min_child_weight" : [1,3,6],
              "n_estimators": [100,200,300],
              "learning_rate": [0.01,0.05,0.1]
             }
t0 = time.time()
grid_search = GridSearchCV(model, param_grid=param_lst, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)
print('gridsearch for xgb spend', time.time()-t0, 'seconds.')
print(grid_search.best_estimator_)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
gridsearch for xgb spend 183.54048657417297 seconds.
XGBClassifier(learning_rate=0.05, max_depth=7, min_child_weight=3)


In [ ]:
### Bayesian Opt
from bayes_opt import BayesianOptimization
from tqdm import tqdm

def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)


    cv_result = xgb.cv(params, dtrain, num_boost_round=num_rounds, nfold=5,
             seed=10,
             callbacks=[xgb.callback.early_stop(50)])

    return cv_result['test-auc-mean'].values[-1]
res = xgb_evaluate(3, 0.5, 4, 0.6, 1, 1)
num_rounds = 3000
random_state = 2021
num_iter = 25
init_points = 5
params = {
    'eta': 0.1,
    'silent': 1,
    'eval_metric': 'auc',
    'verbose_eval': True,
    'seed': random_state
}

xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (0.1, 1),
                                            'max_depth': (5, 15),
                                            'subsample': (0.5, 1),
                                            'gamma': (0, 10),
                                            'alpha': (0, 10),
                                            })

xgbBO.maximize(init_points=init_points, n_iter=num_iter)

In [ ]:
b=12
c=23
d=34
d=23
c=334
o=234455677
dd=1
w=4
pp=7

TypeError: ignored